## Github Miner 
> Module to extract issues from github open source repositories

In [ ]:
#| default_exp miner


In [7]:
#| export
import requests
import json
from urllib.parse import urlencode
from datetime import datetime, timedelta
import pandas as pd

ModuleNotFoundError: No module named 'requests'

In [4]:
#| export
def search_github_issues(query, headers, GITHUB_API_URL, page=1, per_page=10):
    """Search issues across repositories on GitHub."""
    encoded_query = urlencode({"q": query})
    url = f"{GITHUB_API_URL}?{encoded_query}&page={page}&per_page={per_page}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error: {response.status_code}, {response.text}")

In [5]:
#| export
def extract_issue_data(issues):
    """Extract relevant data from issues."""
    extracted_data = []
    for issue in issues:
        issue_data = {
            "id": issue['id'],
            "state" : issue['state'],
            "title": issue['title'],
            "url": issue['html_url'],
            "body": issue['body'],
            "repository_url": issue['repository_url'],
            "pull_request" : issue['pull_request']['url'] if 'pull_request' in  issue else None,
            "comments_url" : issue['comments_url'],
            "labels" : [{'name': label['name'], 'description': label['description']} for label in issue['labels']]
        }
        extracted_data.append(issue_data)
    return extracted_data

In [6]:
#| export
def date_range(start_date, end_date, delta=timedelta(days=365)):
    """Generate date ranges."""
    current_date = start_date
    while current_date < end_date:
        yield current_date, min(current_date + delta, end_date)
        current_date += delta

NameError: name 'timedelta' is not defined

In [ ]:
#| export
def save_data_to_json(data, filename):
    """Save data to a JSON file."""
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
#| export
def load_data_from_json(filename):
    """Load data from a JSON file."""
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        return []  # Return an empty list if the file does not exist
    except json.JSONDecodeError:
        return []  # Return an empty list if the file is empty or has invalid JSON

In [ ]:
#| export
def append_new_data_to_json(new_data, filename):
    """Append new data to an existing JSON file."""
    existing_data = load_data_from_json(filename)
    existing_data.extend(new_data)
    save_data_to_json(existing_data, filename)

In [ ]:
#| export
def fetch_data_and_get_results_by_count(query: str, HEADERS, GITHUB_API_URL):
    """fetch issues matching the query and returns the max possible number of issues"""
    PER_PAGE = 100  # Max is 100
    all_issues = []
    current_page = 1
    total_count = None

    while True:
        try:
            results = search_github_issues(query, HEADERS, page=current_page, per_page=PER_PAGE, GITHUB_API_URL)
            if total_count is None:
                total_count = results['total_count']
                print(f"Total issues to fetch: {total_count}")
            issues = results.get("items", [])
            if not issues:
                break
            all_issues.extend(extract_issue_data(issues))
            print(f"Page {current_page}: {len(issues)} issues fetched")
            if len(all_issues) >= total_count or len(issues) < PER_PAGE:
                break
            current_page += 1
        except Exception as e:
            print(f"An error occurred: {e}")
            break


    print(f"Total issues fetched: {len(all_issues)}")
    return all_issues


